# Project 1
### Epidemiological Study: US Vaccination Campaign (November 2020 - March 2021)
---
### Project Description/Outline
Determine the effectiveness of the US vaccination campaign in attending the population that is most affected by covid-19 disease. This is achieved by revising the total number of doses applied, vaccination coverage, population demographics such as gender, race, socioeconomic status, and education, versus epidemiologic variables: incidence, prevalence, hospitalization, UCI, death numbers.

In [4]:
## Dependencies
import pandas as pd



### Covid-19 Vaccination Data

In [6]:
## Import data from CSV
hesitancy_df = pd.read_csv("data/Vaccine_Hesitancy_Covid19.csv")
hesitancy_df.head()

,FIPS Code,County Name,State,Estimated hesitant,Estimated strongly hesitant,Social Vulnerability Index (SVI),SVI Category,Ability to handle a COVID-19 outbreak (CVAC),CVAC Category,Percent adults fully vaccinated against COVID-19,Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,Percent non-Hispanic Native Hawaiian/Pacific Islander,Percent non-Hispanic White,Geographical Point,State Code,County Boundary,State Boundary
0,1123,"Tallapoosa County, Alabama",ALABAMA,0.23,0.12,0.89,Very High Vulnerability,0.64,High Vulnerability,0.161,0.0242,0.0022,0.0036,0.2697,0.0000,0.6887,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-85.841259 33.104456, -85.8409...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
1,1121,"Talladega County, Alabama",ALABAMA,0.23,0.11,0.87,Very High Vulnerability,0.84,Very High Vulnerability,0.133,0.0229,0.0043,0.0061,0.3237,0.0003,0.6263,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-86.303069 33.46316, -86.30306...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
2,1131,"Wilcox County, Alabama",ALABAMA,0.23,0.11,0.93,Very High Vulnerability,0.94,Very High Vulnerability,0.228,0.0053,0.0009,0.0003,0.6938,0.0000,0.2684,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.52534299999999 32.132773, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
3,1129,"Washington County, Alabama",ALABAMA,0.23,0.11,0.73,High Vulnerability,0.82,Very High Vulnerability,0.192,0.0146,0.0731,0.0025,0.2354,0.0000,0.6495,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-88.45317899999999 31.505388, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
4,1133,"Winston County, Alabama",ALABAMA,0.22,0.11,0.70,High Vulnerability,0.80,High Vulnerability,0.085,0.0315,0.0034,0.0016,0.0073,0.0005,0.9370,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.63656399999999 34.120908, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."


In [7]:
hesitancy_df.iloc[:,10:16].sum(axis=1).mean()*100

97.89445257797577

In [108]:
columns = [
    "FIPS Code",
    "Social Vulnerability Index (SVI)",
    "SVI Category",
    "Percent adults fully vaccinated against COVID-19",
    "Percent Hispanic",
    "Percent non-Hispanic American Indian/Alaska Native",
    "Percent non-Hispanic Asian",
    "Percent non-Hispanic Black",
    "Percent non-Hispanic Native Hawaiian/Pacific Islander",
    "Percent non-Hispanic White"
]

vaccination_df = hesitancy_df[columns]
vaccination_df.head()

,FIPS Code,Social Vulnerability Index (SVI),SVI Category,Percent adults fully vaccinated against COVID-19,Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,Percent non-Hispanic Native Hawaiian/Pacific Islander,Percent non-Hispanic White
0,1123,0.89,Very High Vulnerability,0.161,0.0242,0.0022,0.0036,0.2697,0.0000,0.6887
1,1121,0.87,Very High Vulnerability,0.133,0.0229,0.0043,0.0061,0.3237,0.0003,0.6263
2,1131,0.93,Very High Vulnerability,0.228,0.0053,0.0009,0.0003,0.6938,0.0000,0.2684
3,1129,0.73,High Vulnerability,0.192,0.0146,0.0731,0.0025,0.2354,0.0000,0.6495
4,1133,0.70,High Vulnerability,0.085,0.0315,0.0034,0.0016,0.0073,0.0005,0.9370


### Pendiente
- Calcular y agregar Other

In [19]:
len(vaccination_df)

3142

### US Census Reference (2019)

In [24]:
census_df = pd.read_csv("data/US_Census2019.csv")
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
0,40,3,6,1,0,Alabama,Alabama,4779736,4780125,4785437,...,1.917501,0.578434,1.186314,1.522549,0.563489,0.626357,0.745172,1.090366,1.773786,2.483744
1,50,3,6,1,1,Alabama,Autauga County,54571,54597,54773,...,4.847310,6.018182,-6.226119,-3.902226,1.970443,-1.712875,4.777171,0.849656,0.540916,4.560062
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183112,...,24.017829,16.641870,17.488579,22.751474,20.184334,17.725964,21.279291,22.398256,24.727215,24.380567
3,50,3,6,1,5,Alabama,Barbour County,27457,27455,27327,...,-5.690302,0.292676,-6.897817,-8.132185,-5.140431,-15.724575,-18.238016,-24.998528,-8.754922,-5.165664
4,50,3,6,1,7,Alabama,Bibb County,22915,22915,22870,...,1.385134,-4.998356,-3.787545,-5.797999,1.331144,1.329817,-0.708717,-3.234669,-6.857092,1.831952


In [21]:
census_df.iloc[1:5,15:23].head()

,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,NPOPCHG_2010,NPOPCHG_2011,NPOPCHG_2012,NPOPCHG_2013
1,55243,55390,55533,55869,176,454,-273,-227
2,207601,212521,217855,223234,847,3446,3587,4740
3,25806,25157,24872,24686,-128,14,-172,-232
4,22586,22550,22367,22394,-45,-125,-78,-146


In [25]:
county_pop_df = census_df[["STATE", "COUNTY","STNAME","CTYNAME", "POPESTIMATE2019"]]
# str(census_df['STATE']) + str(census_df['COUNTY'])

In [26]:
county_pop_df.head()

,STATE,COUNTY,STNAME,CTYNAME,POPESTIMATE2019
0,1,0,Alabama,Alabama,4903185
1,1,1,Alabama,Autauga County,55869
2,1,3,Alabama,Baldwin County,223234
3,1,5,Alabama,Barbour County,24686
4,1,7,Alabama,Bibb County,22394


### Pendiente
- Separar County=0 en df aparte
- Armar y agregar County_FIPS

In [28]:
county_pop_df['POPESTIMATE2019'].sum()

656479046

### Covid-19 Case Surveillance


In [ ]:
cases = pd.read_csv("data/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv", chunksize=1000)

In [ ]:
cases1 = pd.DataFrame(cases)

In [86]:
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cdc.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cdc.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("n8mc-b4w4", limit=1000000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [87]:
results_df.tail()

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,case_positive_specimen,underlying_conditions_yn,case_onset_interval
999995,2020-04,NY,36,NASSAU,36059,18 to 49 years,Male,Unknown,Non-Hispanic/Latino,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,0.0,NaN,NaN
999996,2020-04,NY,36,NASSAU,36059,18 to 49 years,Male,Unknown,Non-Hispanic/Latino,Missing,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,No,2.0,NaN,0.0
999997,2020-04,NY,36,NASSAU,36059,18 to 49 years,Male,Unknown,Non-Hispanic/Latino,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,0.0,NaN,NaN
999998,2020-04,NY,36,NASSAU,36059,18 to 49 years,Male,Unknown,Non-Hispanic/Latino,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,No,0.0,NaN,0.0
999999,2020-04,NY,36,NASSAU,36059,18 to 49 years,Male,Unknown,Non-Hispanic/Latino,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,No,0.0,NaN,NaN


In [89]:
len(results_df['case_month'].unique())

16

In [90]:
results_df.sort_values(['case_month'], ascending=True)

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,case_positive_specimen,underlying_conditions_yn,case_onset_interval
341787,2020-01,NJ,34,HUDSON,34017,65+ years,Female,NA,NA,Missing,Missing,Laboratory-confirmed case,Missing,Yes,Missing,No,NaN,NaN,NaN
50334,2020-01,NJ,34,MERCER,34021,50 to 64 years,NA,NA,NA,Missing,Missing,Laboratory-confirmed case,Missing,Yes,Missing,No,NaN,NaN,NaN
50335,2020-01,NJ,34,MERCER,34021,50 to 64 years,NA,NA,NA,Missing,Missing,Laboratory-confirmed case,Missing,Yes,Missing,No,NaN,NaN,NaN
50336,2020-01,NJ,34,MERCER,34021,50 to 64 years,NA,NA,NA,Missing,Missing,Laboratory-confirmed case,Missing,Yes,Missing,No,NaN,NaN,NaN
50337,2020-01,NJ,34,MERCER,34021,50 to 64 years,NA,NA,NA,Missing,Missing,Laboratory-confirmed case,Missing,Yes,Missing,No,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167917,2021-04,NY,36,KINGS,36047,18 to 49 years,Female,Unknown,Unknown,Missing,Missing,Probable Case,Symptomatic,Missing,Missing,Missing,0.0,NaN,0.0
167918,2021-04,NY,36,KINGS,36047,18 to 49 years,Female,Unknown,Unknown,Missing,Missing,Laboratory-confirmed case,Asymptomatic,Missing,Missing,Missing,0.0,NaN,NaN
752558,2021-04,PA,42,CHESTER,42029,50 to 64 years,Female,White,Unknown,Missing,Missing,Laboratory-confirmed case,Unknown,Unknown,Unknown,Unknown,0.0,NaN,NaN
167912,2021-04,NY,36,KINGS,36047,18 to 49 years,Female,Unknown,Unknown,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,0.0,NaN,NaN


In [105]:
results_df['sex'].loc[results_df['ethnicity'] != "Hispanic/Latino"].value_counts()

Female     425273
Male       366967
NA          48979
Unknown      9864
Missing      1737
Name: sex, dtype: int64

In [107]:
results_df['age_group'].value_counts()

18 to 49 years    516724
50 to 64 years    209738
65+ years         125293
0 - 17 years      120874
NA                 17972
Missing             9399
Name: age_group, dtype: int64

In [104]:
columns = [
    "county_fips_code",
    "case_month",
    "current_status",
    "sex",
    "age_group",
    "race",
    "ethnicity",
    "hosp_yn",
    "icu_yn",
    "death_yn"
]

cases_df = results_df[columns]
cases_df.tail()

,county_fips_code,case_month,current_status,sex,age_group,race,ethnicity,hosp_yn,icu_yn,death_yn
999995,36059,2020-04,Laboratory-confirmed case,Male,18 to 49 years,Unknown,Non-Hispanic/Latino,Missing,Missing,No
999996,36059,2020-04,Laboratory-confirmed case,Male,18 to 49 years,Unknown,Non-Hispanic/Latino,No,Missing,No
999997,36059,2020-04,Laboratory-confirmed case,Male,18 to 49 years,Unknown,Non-Hispanic/Latino,Missing,Missing,No
999998,36059,2020-04,Laboratory-confirmed case,Male,18 to 49 years,Unknown,Non-Hispanic/Latino,Missing,Missing,No
999999,36059,2020-04,Laboratory-confirmed case,Male,18 to 49 years,Unknown,Non-Hispanic/Latino,No,Missing,No


### Pendientes

- Definir fecha de corte (28/4/2021 = vaccination)
- Cambiar "case_month" a timestamp
- Eliminar "probable case"
- Nomenclatura AGEGRP de US Census a age_group
- Nomenclatura race/ethnicity combined
- Contar número de líneas del CSV Original
- Bajar TODA la información